## Let's see if we can wire this up to an openbb function
(Even if it's hardcoded for now)


- Agent
- Tools
- Prompt
- AgentExecutor (runtime)


In [4]:
import inspect

from openbb import obb
from openbb_core.app.router import CommandMap
from openbb_core.app.provider_interface import ProviderInterface

In [5]:
from langchain.chat_models import ChatOpenAI
from langchain.agents import tool, Tool, AgentExecutor, AgentOutputParser
from langchain.agents.format_scratchpad import format_to_openai_function_messages
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser

from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.tools.render import format_tool_to_openai_function
from langchain.tools import StructuredTool

In [6]:
from pydantic.v1 import create_model, BaseModel
from pydantic.v1.fields import FieldInfo
from pydantic_core import PydanticUndefinedType

Goal: let's make it easy to convert an OpenBB function --> Langchain / openai function.

Schema comes from the ProviderInterface  
Function / Endpoint map comes from CommandMap + scraping the Members

In [8]:
# All the magic lives in here
from utils import map_openbb_collection_to_langchain_tools
fundamental_tools = map_openbb_collection_to_langchain_tools("/equity/fundamental")

In [181]:
# Let's add some memory
MEMORY_KEY = "chat_history"

# Create our prompt
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You're a powerful financial advisor.
            Do your best to answer the questions,
            and make use of the tools provided, paying special attention to the outputs they provide."""
        ),
        MessagesPlaceholder(variable_name=MEMORY_KEY),
        (
            "user",
            "{input}",
        ),
        MessagesPlaceholder(variable_name="agent_scratchpad")
    ]
)

# Bind our tools
llm = ChatOpenAI(model="gpt-4-1106-preview")
llm_with_tools = llm.bind(functions=[format_tool_to_openai_function(t) for t in compare_tools])  # Never forget!


# Create the agent
agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_function_messages(
            x["intermediate_steps"]
        ),
        "chat_history": lambda x: x["chat_history"]
    }
    | prompt
    | llm_with_tools
    | OpenAIFunctionsAgentOutputParser()
)

agent_executor = AgentExecutor(agent=agent, tools=compare_tools, verbose=True)

In [183]:
type(agent)

langchain.schema.runnable.base.RunnableSequence

In [156]:
OpenAIFunctionsAgentOutputParser?

Init signature: OpenAIFunctionsAgentOutputParser() -> None
Docstring:     
Parses a message into agent action/finish.

Is meant to be used with OpenAI models, as it relies on the specific
function_call parameter from OpenAI to convey what tools to use.

If a function_call parameter is passed, then that is used to get
the tool and tool input.

If one is not passed, then the AIMessage is assumed to be the final output.
Init docstring:
Create a new model by parsing and validating input data from keyword arguments.

Raises ValidationError if the input data cannot be parsed to form a valid model.
File:           ~/miniforge3/envs/ds/lib/python3.10/site-packages/langchain/agents/output_parsers/openai_functions.py
Type:           ModelMetaclass
Subclasses:     

In [ ]:
chat_history = []

In [ ]:
from langchain.schema.messages import AIMessage, HumanMessage

human_input = "List the competitors for TSLA. Use tools."

response = agent_executor.invoke(
    {
        "input": human_input,
        "chat_history": chat_history
    }
    
)

chat_history.extend(
    [
        HumanMessage(content=human_input),
        AIMessage(content=response["output"])
    ]
)



> Entering new AgentExecutor chain...

Invoking: `peers` with `{'symbol': 'TSLA'}`


peers_list=['XPEV', 'LI', 'RIVN', 'LCID', 'GM', 'NIO', 'F', 'FSR', 'MULN']The competitors for Tesla, Inc. (TSLA) include various companies in the electric vehicle (EV) and broader automotive industry. Here's a list of some of the competitors:

1. XPeng Inc. (XPEV) - A Chinese electric vehicle manufacturer.
2. Li Auto Inc. (LI) - Another Chinese company that specializes in electric vehicles.
3. Rivian Automotive, Inc. (RIVN) - An American electric vehicle automaker that focuses on trucks and SUVs.
4. Lucid Group, Inc. (LCID) - An American EV manufacturer that produces luxury electric vehicles.
5. General Motors Company (GM) - A traditional automotive company based in the United States that is expanding into the EV market.
6. NIO Inc. (NIO) - A Chinese electric vehicle manufacturer that produces smart, connected EVs.
7. Ford Motor Company (F) - An established global automotive manufacturer that's incre

In [ ]:
print(response['output'])

The competitors for Tesla, Inc. (TSLA) include various companies in the electric vehicle (EV) and broader automotive industry. Here's a list of some of the competitors:

1. XPeng Inc. (XPEV) - A Chinese electric vehicle manufacturer.
2. Li Auto Inc. (LI) - Another Chinese company that specializes in electric vehicles.
3. Rivian Automotive, Inc. (RIVN) - An American electric vehicle automaker that focuses on trucks and SUVs.
4. Lucid Group, Inc. (LCID) - An American EV manufacturer that produces luxury electric vehicles.
5. General Motors Company (GM) - A traditional automotive company based in the United States that is expanding into the EV market.
6. NIO Inc. (NIO) - A Chinese electric vehicle manufacturer that produces smart, connected EVs.
7. Ford Motor Company (F) - An established global automotive manufacturer that's increasing its investment in electric vehicles.
8. Fisker Inc. (FSR) - An American electric vehicle company that is developing electric passenger vehicles.
9. Mullen 

# Let's explore different agents

In [66]:
from langchain import hub
from langchain.tools.render import render_text_description_and_args
from langchain.agents.format_scratchpad import format_log_to_str # Used for ReAct framework
from langchain.agents.output_parsers import JSONAgentOutputParser
from langchain.agents import AgentExecutor

def langchain_react_agent(tools):
    prompt = hub.pull("hwchase17/react-multi-input-json")
    prompt = prompt.partial(
        tools=render_text_description_and_args(tools),
        tool_names=", ".join([t.name for t in tools])
    )

    llm = ChatOpenAI(model="gpt-4-1106-preview").bind(stop=["\nObservation"])

    chain = (
        {
            "input": lambda x: x["input"],
            "agent_scratchpad": lambda x: format_log_to_str(x["intermediate_steps"])
        }
        | prompt
        | llm
        | JSONAgentOutputParser()
    )

    agent_executor = AgentExecutor(agent=chain, tools=tools, verbose=True)
    return agent_executor

agent_executor=langchain_react_agent(tools=fundamental_tools[:5])

In [69]:
agent_executor.invoke({"input": "How much cash has AAPL had on hand over the years?"})



> Entering new AgentExecutor chain...
Action:
```json
{
  "action": "balance",
  "action_input": {
    "symbol": "AAPL",
    "limit": 5
  }
}
```[FMPBalanceSheetData(symbol=AAPL, date=2023-09-30, cik=0000320193, currency=USD, filling_date=None, accepted_date=2023-11-02 18:08:27, period=FY, cash_and_cash_equivalents=29965000000.0, short_term_investments=31590000000.0, long_term_investments=100544000000.0, inventory=6331000000.0, net_receivables=60985000000.0, marketable_securities=None, property_plant_equipment_net=43715000000.0, goodwill=0.0, assets=352583000000.0, current_assets=143566000000.0, other_current_assets=14695000000.0, intangible_assets=0.0, tax_assets=0.0, non_current_assets=209017000000.0, other_non_current_assets=64758000000.0, account_payables=62611000000.0, tax_payables=0.0, deferred_revenue=8061000000.0, other_assets=0.0, total_assets=None, long_term_debt=95281000000.0, short_term_debt=15807000000.0, liabilities=290437000000.0, other_current_liabilities=58829000000.

{'input': 'How much cash has AAPL had on hand over the years?',
 'output': 'The cash on hand for Apple Inc. (AAPL) over the past five fiscal years is as follows:\n- FY 2023: $29.965 billion\n- FY 2022: $23.646 billion\n- FY 2021: $34.94 billion\n- FY 2020: $38.016 billion\n- FY 2019: $48.844 billion\nThis data represents the cash and cash equivalents reported in the annual balance sheet for each respective fiscal year.'}